<a href="https://colab.research.google.com/github/bytebuster21/AI-project/blob/main/Pre_trained_CNN_model_as_a_Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16 # Import VGG16
from tensorflow.keras import models
import os
import shutil

from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

# --- 2. Define your dataset paths ---
trainpath = '/content/train'
testpath = '/content/test'


if not os.path.exists(trainpath):
    print("Creating dummy dataset for demonstration...")
    os.makedirs(os.path.join(trainpath, 'class_A'), exist_ok=True)
    os.makedirs(os.path.join(trainpath, 'class_B'), exist_ok=True)
    os.makedirs(os.path.join(trainpath, 'class_C'), exist_ok=True)

    os.makedirs(os.path.join(testpath, 'class_A'), exist_ok=True)
    os.makedirs(os.path.join(testpath, 'class_B'), exist_ok=True)
    os.makedirs(os.path.join(testpath, 'class_C'), exist_ok=True)

    from PIL import Image
    def create_dummy_image(path, color, size=(299, 299)):
        img = Image.new('RGB', size, color=color)
        img.save(path)

    # Create dummy training images
    for i in range(116): # 116 * 3 classes = 348
        create_dummy_image(os.path.join(trainpath, 'class_A', f'imgA_{i:03d}.png'), (255, 100, 100))
        create_dummy_image(os.path.join(trainpath, 'class_B', f'imgB_{i:03d}.png'), (100, 255, 100))
        create_dummy_image(os.path.join(trainpath, 'class_C', f'imgC_{i:03d}.png'), (100, 100, 255))

    # Create dummy testing images
    for i in range(51): # 51 * 3 classes = 153
        create_dummy_image(os.path.join(testpath, 'class_A', f'testA_{i:03d}.png'), (200, 50, 50))
        create_dummy_image(os.path.join(testpath, 'class_B', f'testB_{i:03d}.png'), (50, 200, 50))
        create_dummy_image(os.path.join(testpath, 'class_C', f'testC_{i:03d}.png'), (50, 50, 200))

    print(f"Dummy dataset created at {trainpath} and {testpath}")
else:
    print(f"Using existing dataset at {trainpath} and {testpath}")


# --- 4. Configure ImageDataGenerator instances ---
# VGG16 can accept various sizes, 299x299 is specified for this setup
TARGET_SIZE = (299, 299)
BATCH_SIZE = 20 # Not directly used for base model summary but needed for data flow

# Training Data Generator (with augmentation) - still needed for class inference
train_datagen = ImageDataGenerator(rescale=1./255) # Only rescale needed for class_mode inference

# Test Data Generator (ONLY rescaling) - still needed for class inference
test_datagen = ImageDataGenerator(rescale=1./255)

# --- 5. Flow Images from Directories ---
print("\nLoading images using ImageDataGenerator to infer classes...")

train_generator = train_datagen.flow_from_directory(
    trainpath,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE, # Can be 1 for just inferring classes
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    testpath,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE, # Can be 1 for just inferring classes
    class_mode='categorical',
    shuffle=False
)

# Get class names and number of classes from the generator
num_classes = train_generator.num_classes # Number of classes inferred from directory structure
class_names = list(train_generator.class_indices.keys())

print(f"Found {train_generator.samples} training images belonging to {num_classes} classes.")
print(f"Found {test_generator.samples} test images belonging to {num_classes} classes.")
print("Class Names:", class_names)


base_model = VGG16(
    weights='imagenet',        # Use weights pre-trained on ImageNet
    include_top=False,         # Exclude the classifier layers
    input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3) # Define the input image shape
)

base_model.trainable = False

# Print the summary of the pre-trained base model
print("\n--- VGG16 Base Model Summary (Feature Extractor) ---")
base_model.summary()



Mounted at /content/drive
Google Drive mounted successfully!
Creating dummy dataset for demonstration...
Dummy dataset created at /content/train and /content/test

Loading images using ImageDataGenerator to infer classes...
Found 348 images belonging to 3 classes.
Found 153 images belonging to 3 classes.
Found 348 training images belonging to 3 classes.
Found 153 test images belonging to 3 classes.
Class Names: ['class_A', 'class_B', 'class_C']
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- VGG16 Base Model Summary (Feature Extractor) ---


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 299, 299, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 299, 299, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 149, 149, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 149, 149, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 149, 149, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 74, 74, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 74, 74, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 74, 74, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 74, 74, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 37, 37, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 37, 37, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 37, 37, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 37, 37, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 18, 18, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)